In [1]:
import numpy
import pandas
import glob
import sys
import os

sys.path.append(os.path.dirname(os.path.realpath(".")))

from helpers import *

/opt/miniforge3/envs/hmclab-dev/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.0-beta-1-7-g3f59f5a is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
df = pandas.read_csv("xyz_polar.csv", delimiter=",")
channel = df["channels"].values

In [3]:
csv_files = glob.glob("*.picks.csv")
picks_collection = [
    pandas.read_csv(
        csv_file,
        index_col=0,
        names=["channel", "pick", "uncertainty", "snr"],
        skiprows=1,
    )
    for csv_file in csv_files
]

numpy.save("csv_files.npy", csv_files)

Only keep data points that contain values for all columns. i.e. no missing uncertainty or SNR

In [4]:
picks_without_nans_collection = [
    pick[~rows_contain_nans(pick)] for pick in picks_collection
]

In [5]:
n_events = len(picks_without_nans_collection)
observable_arrivals_per_event = len(channel)

data_array = numpy.empty((observable_arrivals_per_event, n_events))
data_array[:] = numpy.nan

uncertainty_array = numpy.empty((observable_arrivals_per_event, n_events))
uncertainty_array[:] = numpy.nan

snr_array = numpy.empty((observable_arrivals_per_event, n_events))
snr_array[:] = numpy.nan

for i_pick, pick in enumerate(picks_without_nans_collection):
    data_array[match_arrays(channel, pick.channel.to_list()), i_pick] = (
        pick.pick - pick.pick.min()
    ) / 200

    uncertainty_array[
        match_arrays(channel, pick.channel.to_list()), i_pick
    ] = pick.uncertainty

    snr_array[match_arrays(channel, pick.channel.to_list()), i_pick] = pick.snr

In [6]:
numpy.save("data_array.npy", data_array)
numpy.save("uncertainty_array.npy", uncertainty_array)
numpy.save("snr_array.npy", snr_array)